In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv("data/train.csv", index_col = "Id")
test_data = pd.read_csv("data/test.csv", index_col = "Id")

Так как метрика в задаче — RMSE от логарифма цены, сразу заменим цену на ее логарифм, и будем оптимизировать RMSE.

In [ ]:
train_data["SalePrice"] = train_data["SalePrice"].apply(np.log)

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

X = train_data.drop(columns = "SalePrice")
y = train_data["SalePrice"]

X_test = test_data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Препроцессинг
__common_transform — заменяет текстовые обозначения для качества на оценки от 1 до 5, чтобы модель понимала, что на этом признаке есть порядок. 

Далее выделяем категориальные и числовые признаки, и группируем категориальные признаки, частота которых меньше 2%.

Пропуски в категориальных признаках заменяяем на "None" — это будет отдельная категория.

In [ ]:
class Preprocessor():
    def __init__(self):
        self.dict = {}
        self.categorical_cols = []
        self.numeric_cols = []

    def __common_transform(self, data):
        qual = {
            "Po": 1,
            "Fa": 2,
            "TA": 3,
            "Gd": 4,
            "Ex": 5
        }
        data["ExterQual"] = data["ExterQual"].map(qual)
        data["ExterCond"] = data["ExterCond"].map(qual)
        data["BsmtQual"] = data["BsmtQual"].map(qual)
        data["BsmtCond"] = data["BsmtCond"].map(qual)
        data["HeatingQC"] = data["HeatingQC"].map(qual)
        data["KitchenQual"] = data["KitchenQual"].map(qual)
        data["FireplaceQu"] = data["FireplaceQu"].map(qual)
        data["GarageQual"] = data["GarageQual"].map(qual)
        data["GarageCond"] = data["GarageCond"].map(qual)

    def fit_transform(self, data):
        self.__common_transform(data)
        threshold = 0.02 * len(data)
        self.categorical_cols = data.select_dtypes(include = ["object"]).columns.to_list()
        self.categorical_cols.append("MSSubClass")
        self.numeric_cols = data.select_dtypes(include = ["number"]).columns.to_list()
        self.numeric_cols.remove("MSSubClass")
        
        for col in self.categorical_cols:
            counts = data[col].value_counts()
            rare = counts[counts < threshold].index.to_list()
            self.dict[col] = rare
            data[col] = data[col].replace(self.dict[col], "Other")
        data[self.categorical_cols] = data[self.categorical_cols].fillna("None")
        

    def transform(self, data):
        self.__common_transform(data)
        data[self.categorical_cols] = data[self.categorical_cols].fillna("None")
        for col in self.categorical_cols:
            data[col] = data[col].replace(self.dict[col], "Other")   

        

In [ ]:
prep = Preprocessor()
prep.fit_transform(X_train)
prep.transform(X_val)
prep.transform(X_test)

# Обучаем модель
Я пробовала разные learning_rate, depth, l2_leaf_reg, но скор на валидации был примерно одинаковым

In [ ]:
from sklearn.metrics import mean_squared_error
model = CatBoostRegressor(
    iterations = 500,
    learning_rate = 0.1,
    depth = 4,
    l2_leaf_reg = 5,
    loss_function = "RMSE",
    eval_metric = "RMSE",
    cat_features = prep.categorical_cols,
    random_seed = 42
)

model.fit(X_train, y_train, eval_set = (X_val, y_val), early_stopping_rounds = 50, use_best_model = True, verbose = 0)
print("Лучшая итерация:", model.get_best_iteration())
y_pred = model.predict(X_val)
print("Score на валидационном наборе: ", np.sqrt(mean_squared_error(y_pred, y_val)))

In [ ]:
import matplotlib.pyplot as plt
evals = model.get_evals_result()
plt.plot(evals["learn"]["RMSE"], label = "RMSE на обучающей выборке")
plt.plot(evals["validation"]["RMSE"], label = "RMSE на валидационной выборке")
plt.xlabel("Итерация")
plt.ylabel("RMSE")
plt.title("Кривая обучения")
plt.axvline(model.get_best_iteration(), linestyle = ":", label = "Лучшая итерация", c = "red")
plt.legend()

# Делаем submit на Kaggle

In [ ]:
y_preds_final = model.predict(X_test)
submission = pd.DataFrame({
    "Id": X_test.index.to_list(),
    "SalePrice": np.exp(y_preds_final)
})

In [ ]:
submission.to_csv("submission.csv", index = False)

# Удаление признаков с низкой важностью
Попробуем улучшить качество с помощью удаления неважных признаков.

Я отбираю признаки, которые имеют важность меньше 2% от максимальной важности.

In [ ]:
feature_importances = model.get_feature_importance()
feature_names = X_train.columns

In [ ]:
fi_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": feature_importances
})
max_importance = fi_df["Importance"].max()
threshold = max_importance * 0.02

In [ ]:
low_imp_f = fi_df[fi_df["Importance"] < threshold]["Feature"].to_list()
low_imp_f

Будем удалять эти признаки по одному и смотреть на скор на валидации.

In [ ]:
models_drop = []
scores = []
for f in low_imp_f:
    X_train_drop = X_train.drop(columns = f)
    X_val_drop = X_val.drop(columns = f)
    cat_features = prep.categorical_cols.copy()
    if f in cat_features:
        cat_features.remove(f)
    model_drop = CatBoostRegressor(
        iterations = 500,
        learning_rate = 0.1,
        depth = 4,
        l2_leaf_reg = 5,
        loss_function = "RMSE",
        eval_metric = "RMSE",
        cat_features = cat_features,
        random_seed = 42,
        verbose = 0
    )
    model_drop.fit(X_train_drop, y_train, eval_set = (X_val_drop, y_val), early_stopping_rounds = 50, use_best_model = True)
    models_drop.append(model_drop)
    y_pred = model_drop.predict(X_val_drop)
    scores.append(np.sqrt(mean_squared_error(y_pred, y_val)))
    print(f"{f}: {scores[-1]}")
    
    

# Делаем submit
Скор улучшился при удалении признаков: SaleType, MasVnrArea, BsmtFinSF2, Heating. Попробуем каждый удалить и отправить на Kaggle

In [ ]:
y_preds_final_st = models_drop[low_imp_f.index("SaleType")].predict(X_test.drop(columns = "SaleType"))
submission_st = pd.DataFrame({
    "Id": X_test.index.to_list(),
    "SalePrice": np.exp(y_preds_final_st)
})

In [ ]:
submission_st.to_csv("submission_st.csv", index = False)

In [ ]:
y_preds_final_mva = models_drop[low_imp_f.index("MasVnrArea")].predict(X_test.drop(columns = "MasVnrArea"))
submission_mva = pd.DataFrame({
    "Id": X_test.index.to_list(),
    "SalePrice": np.exp(y_preds_final_mva)
})

In [ ]:
submission_mva.to_csv("submission_mva.csv", index = False)

In [ ]:
y_preds_final_bfs = models_drop[low_imp_f.index("BsmtFinSF2")].predict(X_test.drop(columns = "BsmtFinSF2"))
submission_bfs = pd.DataFrame({
    "Id": X_test.index.to_list(),
    "SalePrice": np.exp(y_preds_final_bfs)
})

In [ ]:
submission_bfs.to_csv("submission_bfs.csv", index = False)

In [ ]:
y_preds_final_h = models_drop[low_imp_f.index("Heating")].predict(X_test.drop(columns = "Heating"))
submission_h = pd.DataFrame({
    "Id": X_test.index.to_list(),
    "SalePrice": np.exp(y_preds_final_h)
})

In [ ]:
submission_h.to_csv("submission_h.csv", index = False)

# Лучший submit
Лучший скор на тесте получился у модели с удаленным столбцом BsmtFinSF2:

0.12671